In [ ]:
!pip install opencv-python
!pip install pandas
!pip install tensorflow-gpu
!pip install matplotlib

In [2]:
import cv2
import matplotlib.pyplot as plt
import pandas as pd
import os
import tensorflow as tf
from tensorflow.keras import layers

In [16]:
IMAGE_HEIGHT = 64
IMAGE_WIDTH = 64

In [24]:
img_dir = './train/processed_images'
train_data = tf.keras.preprocessing.image_dataset_from_directory(img_dir,                                                                  
    label_mode = "categorical",
    image_size = (IMAGE_HEIGHT,IMAGE_WIDTH),
    batch_size= 128,
    seed = 42,
    validation_split = 0.25,
    subset = "training"
)
test_data = tf.keras.preprocessing.image_dataset_from_directory(img_dir,
    label_mode = "categorical",
    image_size = (IMAGE_HEIGHT,IMAGE_WIDTH),
    batch_size = 128,
    seed = 42,
    validation_split = 0.25,
    subset = "validation"
)

Found 483697 files belonging to 12 classes.
Using 362773 files for training.
Found 483697 files belonging to 12 classes.
Using 120924 files for validation.


In [18]:
# plt.figure(figsize=(16,16))
# for image,label in train_data.take(1):
#   for i in range(18):
#     plt.subplot(6,3,i+1)
#     plt.imshow(image[i].numpy().astype("uint8"))
#     plt.title(f"Age : {train_data.class_names[tf.argmax(label[i])]}")
#     plt.axis("off")
#     plt.tight_layout()

In [30]:
model = tf.keras.Sequential([
    layers.Conv2D(filters = 20, kernel_size = 2,input_shape= (IMAGE_HEIGHT,IMAGE_WIDTH,3),padding="same",activation= "relu"),
    layers.BatchNormalization(),
    layers.MaxPooling2D(pool_size= 2),
    layers.Conv2D(filters = 20, kernel_size = 2, padding= "same",activation= "relu"),
    layers.BatchNormalization(),
    layers.MaxPooling2D(pool_size= 2),
    layers.Conv2D(filters = 20, kernel_size= 2, padding = "same",activation= "relu"),
    layers.BatchNormalization(),
    layers.MaxPooling2D(pool_size = 2),
    layers.Conv2D(filters = 20, kernel_size= 2, padding = "same",activation= "relu"),
    layers.Flatten(),
    layers.Dense(1024, activation = "relu"),                        
    layers.Dropout(0.3),
    layers.Dense(1024, activation = "relu"),                          
    layers.Dropout(0.3),
    layers.Dense(1024, activation = "relu"),
    layers.Dense(12, activation= "softmax")
])
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_16 (Conv2D)          (None, 64, 64, 20)        260       
                                                                 
 batch_normalization_12 (Bat  (None, 64, 64, 20)       80        
 chNormalization)                                                
                                                                 
 max_pooling2d_12 (MaxPoolin  (None, 32, 32, 20)       0         
 g2D)                                                            
                                                                 
 conv2d_17 (Conv2D)          (None, 32, 32, 20)        1620      
                                                                 
 batch_normalization_13 (Bat  (None, 32, 32, 20)       80        
 chNormalization)                                                
                                                      

In [31]:
model.compile(
    loss = tf.keras.losses.categorical_crossentropy,
    optimizer = tf.keras.optimizers.Adam(learning_rate= 0.0001),
    metrics = ["accuracy"]
)

In [33]:
if tf.test.gpu_device_name(): 
    print('Default GPU Device:{}'.format(tf.test.gpu_device_name()))
else:
   print("Please install GPU version of TF")

Default GPU Device:/device:GPU:0


In [32]:
checkpoint_path = './results/all_dataset.h5'

model_checkpoint = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path, 
    verbose=1, 
    save_weights_only=False, 
)

In [34]:
model.fit(
    train_data, 
    steps_per_epoch = len(train_data),      
    epochs = 200,
    validation_data = test_data,
    validation_steps = len(test_data),
    callbacks = [model_checkpoint]    
)

Epoch 1/200
 258/1418 [====>.........................] - ETA: 5:45 - loss: 2.0338 - accuracy: 0.2443

KeyboardInterrupt: 